# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
from collections import Counter

In [2]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [16]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [17]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [18]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [3]:
def foldl2(f, x0, lst):
    #return foldr(lambda x, y: f(y, x), x0, lst[::-1])
    return (foldr(lambda a, g: lambda k: g(f(k,a)), lambda x: x, lst))(x0)

In [4]:
foldl2(lambda x, y: x + y, 0, [1, 2, 3])

6

In [5]:
foldl2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [6]:
def foldr2(f, x0, lst):
    #return foldl(lambda x, y: f(y, x), x0, lst[::-1])
    return (foldl(lambda g, a: lambda k: g(f(a,k)), lambda x: x, lst))(x0)

In [7]:
foldr2(lambda x, y: x + y, 0, [1, 2, 3])

6

In [8]:
foldr2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [58]:
import string

def check_inv(a, b):
    l_a = len(a)
    l_b = len(b)
    if l_b > l_a:
        return False
    alph = dict.fromkeys(string.ascii_letters, 1)
    for i in b:
        alph[i] = 0
    for j in range(l_a - l_b):
        inv = True
        for k in range(j, j + l_b):
            if alph[a[k]] == 1:
                inv = False
                break
        if inv:
            return True
    return False
    
check_inv('akdndixlndxnckmefdbcafjdkvndl', 'abcdef')

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [78]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if(not self.left and not self.right):
            yield self.value
        if(self.left):
            yield from self.left
        if(self.right):
            yield from self.right
            
    
    def __str__(self):
        str_ = [str(self.value)]
        if self.left:
            str_.append(self.left.__str__())
        if self.right:
            str_.append(self.right.__str__())
        return ' '.join(str_)
    
    def __repr__(self):
        if(not self.left and not self.right):
            return f"Tree({self.value})"
        
        if(not self.right):
            return f"Tree({self.value}, {self.left.__repr__()})"
        
        if(not self.left):
            return f"Tree({self.value}, None, {self.right.__repr__()})"
        
        
        return f"Tree({self.value}, {self.left.__repr__()}, {self.right.__repr__()})"
 
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

In [85]:
for i in tree:
    print(i, end=' ')

3 4 2 

In [79]:
print(tree)

0 1 3 4 2


In [76]:
tree

Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [46]:
import operator
OPERATORS = {'+': (1, operator.add), '-': (1, operator.sub),
             '*': (2, operator.mul), '/': (2, operator.truediv)}
def calc(expr):  
    def parse(expr):
        parsed_expr = ''
        for i in expr:
            if i.isnumeric():
                parsed_expr += i  
            elif parsed_expr: 
                yield int(parsed_expr) 
                parsed_expr = ''
            if i in OPERATORS or i in "()": 
                yield i 
        if parsed_expr:  
            yield int(parsed_expr)  
        
    def shunt(parsed_expr):
        stack = []
        for i in parsed_expr:
            if i in OPERATORS: 
                while stack and stack[-1] != "(" and OPERATORS[i][0] <= OPERATORS[stack[-1]][0]:
                    yield stack.pop()
                stack.append(i)
            elif i == ")":
                while stack:
                    x = stack.pop()
                    if x == "(":
                        break
                    yield x
            elif i == "(":
                stack.append(i)
            else:
                yield i
        while stack:
            yield stack.pop()
    def vych(pol_expr):
        stack = []
        for i in pol_expr:
            if i in OPERATORS:  
                y, x = stack.pop(), stack.pop()  
                stack.append(OPERATORS[i][1](x, y)) 
            else:
                stack.append(i)
        return stack[0]     
    return vych(shunt(parse(expr))) 

In [47]:
print(calc('2 * (15 - 3 * 4) - 3'))

3
